## Check if model converges at all

In [1]:
%load_ext autoreload

In [2]:
import os, sys, importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf

BASE_PATH = os.path.join(os.getcwd(), "..", "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DATASETS_PATH = os.path.join(BASE_PATH, "datasets")

sys.path.append(MODULE_PATH)

from active_learning import TrainConfig
from data import BenchmarkData, DataSetType
from models import setup_growth, default_model
from bayesian import BayesModel, McDropout, MomentPropagation

import mp.MomentPropagation as mp

In [3]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [4]:
output_classes = 5

In [5]:
benchmark = BenchmarkData(DataSetType.MNIST, os.path.join(DATASETS_PATH, "mnist"), classes=output_classes)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(benchmark.inputs, benchmark.targets)

In [7]:
print("Datapoints: {}".format(len(benchmark.targets)))
print("Unique labels: {}".format(np.unique(benchmark.targets)))

Datapoints: 35735
Unique labels: [0 1 2 3 4]


In [8]:
model = default_model(output_classes=output_classes)
model.compile(optimizer="adadelta", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [9]:
x_test.shape

(8934, 28, 28, 1)

In [10]:
y_test.shape

(8934,)

In [11]:
model.fit(x_train, y_train, epochs=10, batch_size=100)

Epoch 1/10
269/269 [==============================] - 8s 31ms/step - loss: 1.9962 - accuracy: 0.1898
Epoch 2/10
269/269 [==============================] - 2s 6ms/step - loss: 1.5899 - accuracy: 0.2043
Epoch 3/10
269/269 [==============================] - 2s 6ms/step - loss: 1.5356 - accuracy: 0.2131
Epoch 4/10
269/269 [==============================] - 2s 6ms/step - loss: 1.4901 - accuracy: 0.2185
Epoch 5/10
269/269 [==============================] - 2s 6ms/step - loss: 1.4555 - accuracy: 0.2252
Epoch 6/10
269/269 [==============================] - 2s 6ms/step - loss: 1.4208 - accuracy: 0.2291
Epoch 7/10
269/269 [==============================] - 2s 6ms/step - loss: 1.3884 - accuracy: 0.2417
Epoch 8/10
269/269 [==============================] - 2s 6ms/step - loss: 1.3544 - accuracy: 0.2447
Epoch 9/10
269/269 [==============================] - 2s 6ms/step - loss: 1.3133 - accuracy: 0.2498
Epoch 10/10
269/269 [==============================] - 2s 6ms/step - loss: 1.2763 - accuracy: 0.257

In [12]:
model.evaluate(x_test, y_test)

280/280 [==============================] - 3s 12ms/step - loss: 1.1440 - accuracy: 0.3214


[1.1439809799194336, 0.3213566243648529]

# Model Debug

In [13]:
mc_model = McDropout(model)
mc_model.evaluate(x_test[:100], y_test[:100])

[1.1764604, 0.321]

In [14]:
model.evaluate(x_test[:100], y_test[:100])

4/4 [==============================] - 3s 704ms/step - loss: 1.1248 - accuracy: 0.3400


[1.124780297279358, 0.3400000035762787]

In [15]:
mc_model.fit(x_train, y_train, epochs=100, batch_size=100)

Epoch 1/100
269/269 [==============================] - 2s 6ms/step - loss: 1.2434 - accuracy: 0.2563
Epoch 2/100
269/269 [==============================] - 2s 6ms/step - loss: 1.2162 - accuracy: 0.2617
Epoch 3/100
269/269 [==============================] - 2s 6ms/step - loss: 1.1953 - accuracy: 0.2686
Epoch 4/100
269/269 [==============================] - 2s 6ms/step - loss: 1.1727 - accuracy: 0.2747
Epoch 5/100
269/269 [==============================] - 2s 6ms/step - loss: 1.1525 - accuracy: 0.2832
Epoch 6/100
269/269 [==============================] - 2s 6ms/step - loss: 1.1346 - accuracy: 0.2940
Epoch 7/100
269/269 [==============================] - 2s 6ms/step - loss: 1.1068 - accuracy: 0.3071
Epoch 8/100
269/269 [==============================] - 2s 6ms/step - loss: 1.0648 - accuracy: 0.3227
Epoch 9/100
269/269 [==============================] - 2s 6ms/step - loss: 1.0021 - accuracy: 0.3398
Epoch 10/100
269/269 [==============================] - 2s 6ms/step - loss: 0.9541 - accura

269/269 [==============================] - 2s 6ms/step - loss: 0.1140 - accuracy: 0.9705
Epoch 82/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1017 - accuracy: 0.9717
Epoch 83/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1050 - accuracy: 0.9704
Epoch 84/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1104 - accuracy: 0.9701
Epoch 85/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1043 - accuracy: 0.9719
Epoch 86/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1025 - accuracy: 0.9731
Epoch 87/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1016 - accuracy: 0.9728
Epoch 88/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1024 - accuracy: 0.9719
Epoch 89/100
269/269 [==============================] - 2s 6ms/step - loss: 0.1010 - accuracy: 0.9723
Epoch 90/100
269/269 [==============================] - 2s 6ms/step - loss: 0.0991 - accuracy: 

In [16]:
mc_model.get_model().evaluate(x_test[:100], y_test[:100])

4/4 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 0.9900


[0.014902540482580662, 0.9900000095367432]

In [17]:
mc_model.evaluate(x_test[:100], y_test[:100])

[0.017985443, 0.99]